In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../skl_sampling_bayesian_transformer'))

In [2]:
import pandas as pd 
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# https://github.com/DenisVorotyntsev/CategoricalEncodingBenchmark/blob/master/notebooks/1-prepare-datasets.ipynb

In [16]:
import sampling_bayesian_encoder

In [ ]:
def process_cols(df, cols_dict, sample=None):
    
    if sample is not None:
        n_examples = int(df.shape[0] * sample)
        df = df.sample(n_examples, random_state=42).reset_index(drop=True)
    
    d = {}
    cat_cols = []
    for ind, col in enumerate(df.columns): 
        if col != "target":
            if col in cols_dict["cat_cols"]:
                d[col] = f"cat_{ind}"
                cat_cols.append(f"cat_{ind}")
            else:
                d[col] = f"num_{ind}"
    df = df.rename(d, axis=1)
    for col in cat_cols:
        df[col] = df[col].fillna("new_category_nan")
        df[col] = df[col].astype("str")
    print("Tot cat columns : ", len(cat_cols))
    return df

In [ ]:
dataset_name = "telecom"
data_pth = f"../data/{dataset_name}/"
data = pd.read_csv(f"{data_pth}WA_Fn-UseC_-Telco-Customer-Churn.csv")
print(data.shape)
data.head()

In [ ]:
#https://www.kaggle.com/subinium/11-categorical-encoders-and-benchmark

In [4]:
import pandas as pd

from category_encoders.ordinal import OrdinalEncoder
from category_encoders.woe import WOEEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder


In [5]:
%%time
train = pd.read_csv('csv/input/cat-in-the-dat/train.csv')
test = pd.read_csv('csv/input/cat-in-the-dat/test.csv')
target = train['target']
train_id = train['id']
test_id = test['id']
train.drop(['target', 'id'], axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

Wall time: 1.26 s


In [6]:
feature_list = list(train.columns)

In [ ]:
feature_list

In [ ]:
# Label encoder

In [7]:
%%time
LE_encoder = OrdinalEncoder(feature_list)
train_le = LE_encoder.fit_transform(train)
test_le = LE_encoder.transform(test)

Wall time: 2.02 s


In [ ]:
# One hot encoder RAM ISSUE with cat encoding

In [8]:
#%%time
#OHE_encoder = OneHotEncoder(feature_list)
#train_ohe = OHE_encoder.fit_transform(train)
#test_ohe = OHE_encoder.transform(test)

Wall time: 0 ns


MemoryError: Unable to allocate 4.95 GiB for an array with shape (2215, 300000) and data type int64

In [10]:
%%time

TE_encoder = TargetEncoder()
train_te = TE_encoder.fit_transform(train[feature_list], target)
test_te = TE_encoder.transform(test[feature_list])

train_te.head()

Wall time: 4.03 s


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0.302537,0.290107,0.327145,0.360978,0.307162,0.242813,0.237743,...,0.372694,0.368421,2,0.403885,0.257877,0.306993,0.208354,0.401186,2,2
1,0,1,0,0.302537,0.290107,0.327145,0.290054,0.359209,0.289954,0.304164,...,0.189189,0.076924,1,0.403885,0.326315,0.206599,0.186877,0.303880,7,8
2,0,0,0,0.309384,0.290107,0.241790,0.290054,0.293085,0.289954,0.353951,...,0.223022,0.172414,1,0.317175,0.403126,0.306993,0.351864,0.206843,7,2
3,0,1,0,0.309384,0.290107,0.351052,0.290054,0.307162,0.339793,0.329472,...,0.325123,0.227273,1,0.403885,0.360961,0.330148,0.208354,0.355985,2,1
4,0,0,0,0.309384,0.333773,0.351052,0.290054,0.293085,0.339793,0.329472,...,0.376812,0.200000,1,0.403885,0.225214,0.206599,0.351864,0.404345,7,8


In [ ]:
#M-Estimate Encoder

In [13]:
%%time
MEE_encoder = MEstimateEncoder()
train_mee = MEE_encoder.fit_transform(train[feature_list], target)
test_mee = MEE_encoder.transform(test[feature_list])
train_mee.head()

Wall time: 3.51 s


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0.302537,0.290107,0.327145,0.360976,0.307162,0.242815,0.237744,...,0.372448,0.365294,2,0.403884,0.257879,0.306993,0.208379,0.400998,2,2
1,0,1,0,0.302537,0.290107,0.327145,0.290055,0.359207,0.289954,0.304164,...,0.190231,0.093277,1,0.403884,0.326314,0.206602,0.186884,0.303881,7,8
2,0,0,0,0.309384,0.290107,0.241791,0.290055,0.293085,0.289954,0.353950,...,0.223319,0.176863,1,0.317175,0.403125,0.306993,0.351861,0.206881,7,2
3,0,1,0,0.309384,0.290107,0.351051,0.290055,0.307162,0.339792,0.329472,...,0.325029,0.229020,1,0.403884,0.360960,0.330147,0.208379,0.355965,2,1
4,0,0,0,0.309384,0.333773,0.351051,0.290055,0.293085,0.339792,0.329472,...,0.376471,0.202941,1,0.403884,0.225215,0.206602,0.351861,0.404310,7,8


In [ ]:
# Weight of Evidence Encoder

In [12]:
%%time
WOE_encoder = WOEEncoder()
train_woe = WOE_encoder.fit_transform(train[feature_list], target)
test_woe = WOE_encoder.transform(test[feature_list])
train_woe.head()

Wall time: 3.5 s


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,-0.015794,-0.075416,0.098327,0.248359,0.006058,-0.317803,-0.345614,...,0.302749,0.333932,2,0.430146,-0.237516,0.005297,-0.514545,0.420532,2,2
1,0,1,0,-0.015794,-0.075416,0.098327,-0.075660,0.240683,-0.076148,-0.008087,...,-0.600377,-1.052362,1,0.430146,0.094611,-0.526005,-0.650766,-0.008737,7,8
2,0,0,0,0.016454,-0.075416,-0.323420,-0.075660,-0.060990,-0.076148,0.217747,...,-0.417323,-0.607677,1,0.052724,0.426997,0.005297,0.208660,-0.523234,7,2
3,0,1,0,0.016454,-0.075416,0.205037,-0.075660,0.006058,0.155252,0.108884,...,0.096879,-0.338013,1,0.430146,0.248265,0.111980,-0.514545,0.227089,2,1
4,0,0,0,0.016454,0.128285,0.205037,-0.075660,-0.060990,0.155252,0.108884,...,0.321353,-0.468414,1,0.430146,-0.416062,-0.526005,0.208660,0.432324,7,8


In [19]:
%%time
JSE_encoder = JamesSteinEncoder()
train_jse = JSE_encoder.fit_transform(train[feature_list], target)
test_jse = JSE_encoder.transform(test[feature_list])

Wall time: 3.89 s


In [18]:
%%time
SBE_encoder = sampling_bayesian_encoder.SamplingBayesianEncoder(cols = feature_list)
train_sbe = SBE_encoder.fit_transform(train)
test_sbe = SBE_encoder.transform(test)

TypeError: exceptions must derive from BaseException